In [41]:
import pandas as pd
import numpy as np
import re


In [42]:
df_raw_tbh = pd.read_csv('turnbackhoax_data.csv')

In [44]:
df_raw_tbh['Content']

0       Konten yang beredar merupakan hasil rekayasa A...
1       Video yang beredar adalah hasil rekayasa AI.\n...
2       Faktanya, ledakan hanya mungkin terjadi bila t...
3       Faktanya Kemenaker menyalurkan BSU pada Juni-J...
4       Faktanya, pengecekan BSU melalui Pospay hanya ...
                              ...                        
9995    Hasil Periksa Fakta Khairunnisa Andini (Univer...
9996    Hasil Periksa Fakta Ani Nur MR (Universitas Ai...
9997    Hasil Periksa Fakta Gabriela Nauli Sinaga (Uni...
9998    Hasil periksa fakta Aisyah Adilah (Anggota Kom...
9999    Direktorat Jenderal Imigrasi Kemenkumham menya...
Name: Content, Length: 10000, dtype: object

Berdasarkan struktur narasi TurnBackHoax, kita perlu mengambil bagian narasinya saja.

In [ ]:
def ambil_narasi_aja(full_text):
    narasi=''
    narasi_start_splitter = ['NARASI:', 'Narasi(Diterjemahkan ke Bahasa Indonesia)', '[NARASI]:', 'Narasi :', '[NARASI]', 'Narasi:', 'NARASI :']
    for s in narasi_start_splitter:
        if s in full_text:
            narasi = full_text.split(s)[1]
            break
    narasi_ended = ''
    narasi_end_splitter = ['PENJELASAN:', '[PENJELASAN]:', 'Penjelasan :', '[PENJELASAN]', 'PENJELASAN', 'PENJELASAN :']
    for s in narasi_end_splitter:
        if s in full_text:
            narasi_ended = narasi.split(s)[0]
            break

    if len(narasi_ended) > 0:
        narasi = narasi_ended

    narasi_cont = ''
    narasi_cont_splitter = ['(Lanjutan Narasi)']
    for s in narasi_cont_splitter:
        if s in full_text:
            narasi_cont = full_text.split(s)[1]

    narasi_full = narasi + '' + narasi_cont

    return narasi_full

In [20]:
df_raw_tbh['Clean Narasi'] = df_raw_tbh['Content'].map(lambda x: ambil_narasi_aja(x))
df_raw_tbh['Clean Narasi'] = df_raw_tbh['Clean Narasi'].str.lower()
df_raw_tbh['Title'] = df_raw_tbh['Title'].str.lower()

In [33]:
df_raw_tbh['Clean Narasi'].value_counts()

Clean Narasi
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   4197
 (tidak terdapat narasi)\n===\n                                                                                                                                                                                                                                                                                                                                                                                    

In [35]:
def remove_url(textDisplay):
    if textDisplay is not None and isinstance(textDisplay, str):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'', textDisplay)
    else:
        return textDisplay
    
def remove_html(textDisplay):
    if textDisplay is not None and isinstance(textDisplay, str):
        html = re.compile(r'<.*?>')
        return html.sub(r'', textDisplay)
    else:
        return textDisplay
    
# def remove_emoji(textDisplay):
#     if textDisplay is not None and isinstance(textDisplay, str):

def remove_symbol(textDisplay):
    if textDisplay is not None and isinstance(textDisplay, str):
        textDisplay = re.sub(r'[^a-zA-Z0-9\s]','',textDisplay)
    return textDisplay

In [38]:
df_raw_tbh['Title'] = df_raw_tbh['Title'].apply(lambda x: remove_html(x))
df_raw_tbh['Title'] = df_raw_tbh['Title'].apply(lambda x: remove_symbol(x))
df_raw_tbh['Title'] = df_raw_tbh['Title'].apply(lambda x: remove_url(x))

df_raw_tbh['Clean Narasi'] = df_raw_tbh['Clean Narasi'].apply(lambda x: remove_html(x))
df_raw_tbh['Clean Narasi'] = df_raw_tbh['Clean Narasi'].apply(lambda x: remove_symbol(x))
df_raw_tbh['Clean Narasi'] = df_raw_tbh['Clean Narasi'].apply(lambda x: remove_url(x))

In [39]:
df_raw_tbh.to_csv('dataframe_turnbackhoax.csv', index=False)